# 使用 Neural Network 來預測
---

In [71]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random

In [2]:
# 銀行的檔案
trainFile = open("./Data/bank-full.csv","r") 
testFile = open("./Data/bank.csv","r")

# 吐掉第一行
trainFile.readline()
testFile.readline()


# 資料
trainOrgDataArray = []
testOrgDataArray = []
for line in trainFile:
    lineData = line.split(";")
    
    tempArray = []
    for data in lineData:
        tempArray.append(data.replace("\n",""))
    trainOrgDataArray.append(tempArray)

trainFile.close()
testFile.close()

## 資料愈處理的函數
---

In [3]:
def job(x):
    return {
        "\"admin.\"":         [1,0,0,0,0,0,0,0,0,0,0,0],
        "\"unknown\"":        [0,1,0,0,0,0,0,0,0,0,0,0],
        "\"unemployed\"":     [0,0,1,0,0,0,0,0,0,0,0,0],
        "\"management\"":     [0,0,0,1,0,0,0,0,0,0,0,0],
        "\"housemaid\"":      [0,0,0,0,1,0,0,0,0,0,0,0],
        "\"entrepreneur\"":   [0,0,0,0,0,1,0,0,0,0,0,0],
        "\"student\"":        [0,0,0,0,0,0,1,0,0,0,0,0],
        "\"blue-collar\"":    [0,0,0,0,0,0,0,1,0,0,0,0],
        "\"self-employed\"":  [0,0,0,0,0,0,0,0,1,0,0,0],
        "\"retired\"":        [0,0,0,0,0,0,0,0,0,1,0,0],
        "\"technician\"":     [0,0,0,0,0,0,0,0,0,0,1,0],
        "\"services\"":       [0,0,0,0,0,0,0,0,0,0,0,1]
    }.get(x,                  [1,0,0,0,0,0,0,0,0,0,0,0])

In [4]:
def marital(x):
    return {
        "\"married.\"":       [1,0,0],
        "\"divorced\"":       [0,1,0],
        "\"single\"":         [0,0,1]
    }.get(x,                  [1,0,0])

In [5]:
def education(x):
    return {
        "\"unknown\"":        [1,0,0,0],
        "\"secondary\"":      [0,1,0,0],
        "\"primary\"":        [0,0,1,0],
        "\"tertiary\"":       [0,0,0,1]
    }.get(x,                  [1,0,0,0])

In [6]:
def default(x):
    return {
        "\"yes\"":            [1,0],
        "\"no\"":             [0,1]
    }.get(x,                  [1,0])

In [7]:
def housing(x):
    return {
        "\"yes\"":            [1,0],
        "\"no\"":             [0,1]
    }.get(x,                  [1,0])

In [8]:
def loan(x):
    return {
        "\"yes\"":            [1,0],
        "\"no\"":             [0,1]
    }.get(x,                  [1,0])

In [9]:
def contact(x):
    return {
        "\"unknown\"":        [1,0,0],
        "\"telephone\"":      [0,1,0],
        "\"cellular\"":       [0,0,1]
    }.get(x,                  [1,0,0])

In [10]:
def month(x):
    return {
        "\"jan\"":            [1,0,0,0,0,0,0,0,0,0,0,0],
        "\"feb\"":            [0,1,0,0,0,0,0,0,0,0,0,0],
        "\"mar\"":            [0,0,1,0,0,0,0,0,0,0,0,0],
        "\"apr\"":            [0,0,0,1,0,0,0,0,0,0,0,0],
        "\"may\"":            [0,0,0,0,1,0,0,0,0,0,0,0],
        "\"jun\"":            [0,0,0,0,0,1,0,0,0,0,0,0],
        "\"jul\"":            [0,0,0,0,0,0,1,0,0,0,0,0],
        "\"aug\"":            [0,0,0,0,0,0,0,1,0,0,0,0],
        "\"sep\"":            [0,0,0,0,0,0,0,0,1,0,0,0],
        "\"oct\"":            [0,0,0,0,0,0,0,0,0,1,0,0],
        "\"nov\"":            [0,0,0,0,0,0,0,0,0,0,1,0],
        "\"dec\"":            [0,0,0,0,0,0,0,0,0,0,0,1]
    }.get(x,                  [1,0,0,0,0,0,0,0,0,0,0,0])

In [11]:
def poutcome(x):
    return {
        "\"unknown\"":        [1,0,0,0],
        "\"other\"":          [0,1,0,0],
        "\"failure\"":        [0,0,1,0],
        "\"success\"":        [0,0,0,1]
    }.get(x,                  [1,0,0,0])

In [12]:
def termDeposit(x):
    return {
        "\"yes\"":            1,
        "\"no\"":             0
    }.get(x,                  1)

## 轉乘資料形式
---

In [13]:
# 將某一筆資料，全部轉乘 Training 的資料形式
def TransformDataToTraining(DataList):
    OutputArray = []
    
    OutputArray.extend([int(DataList[0])])
    OutputArray.extend(job(DataList[1]))
    OutputArray.extend(marital(DataList[2]))
    OutputArray.extend(education(DataList[3]))
    OutputArray.extend(default(DataList[4]))
    OutputArray.extend([int(DataList[5])])
    OutputArray.extend(housing(DataList[6]))
    OutputArray.extend(loan(DataList[7]))
    OutputArray.extend(contact(DataList[8]))
    OutputArray.extend([int(DataList[9])])
    OutputArray.extend(month(DataList[10]))
    OutputArray.extend([int(DataList[11])])
    OutputArray.extend([int(DataList[12])])
    OutputArray.extend([int(DataList[13])])
    OutputArray.extend([int(DataList[14])])
    OutputArray.extend(poutcome(DataList[15]))
    
    Ans = termDeposit(DataList[16])
    return OutputArray, Ans

In [14]:
# 要找出最小跟最大的陣列
def FindMinAndMax(DataList):
    OutputArray = []
    
    # 先把每一個的值拿進來
    for i in range(0, len(DataList[0])):
        # 產生暫存的 Array
        tempArray = []
        
        for dataNum in range(0, len(DataList)):
            tempArray.append(DataList[dataNum][i])
        OutputArray.append([min(tempArray), max(tempArray)])
    return OutputArray

In [42]:
# Feature 資料
FeatureDataList = []

# Ans
AnsDataList = []

for i in range(0, len(trainOrgDataArray)):
    FeatureData, AnsData = TransformDataToTraining(trainOrgDataArray[i])
    FeatureDataList.append(FeatureData)
    AnsDataList.append([0, 1] if AnsData == 1 else [1, 0])
    
# 找出最大最小值，為了要 Normalize
MinMaxArrayList = FindMinAndMax(FeatureDataList)
for i in range(0, len(FeatureDataList)):
    for j in range(0, len(FeatureDataList[0])):
        Min = MinMaxArrayList[j][0]
        Max = MinMaxArrayList[j][1]
        FeatureDataList[i][j] = (FeatureDataList[i][j] - Min) / (Max - Min)

In [43]:
# 測試輸入
print("Features")
print(FeatureDataList[0])
print("Label Ans")
print(AnsDataList[0])

Features
[0.5194805194805194, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.09225936484302653, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.13333333333333333, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05307035380235868, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
Label Ans
[1, 0]


## 建構 Tensorflow
---

In [38]:
# Size 設定
input_size = len(FeatureDataList[0])
hidden_size = 500
output_size = 2

print(input_size)

51


In [64]:
# Input Size => 51
x = tf.placeholder(tf.float32, [None, input_size])

# Output Size => 2 (0, 1) 的機率
y = tf.placeholder(tf.float32, [None, output_size])

# 設定中間的  Weight & Bias
hidden_weights = tf.Variable(tf.random_normal([input_size, hidden_size]))
output_weights = tf.Variable(tf.random_normal([hidden_size, output_size]))

hidden_biases = tf.Variable(tf.random_normal([hidden_size]))
output_biases = tf.Variable(tf.random_normal([output_size]))

In [65]:
hidden_layer = tf.matmul(x, hidden_weights) + hidden_biases
hidden_layer = tf.nn.relu(hidden_layer)

output_layer = tf.matmul(hidden_layer, output_weights) + output_biases

In [68]:
cost = tf.losses.softmax_cross_entropy(
    logits = output_layer,
    onehot_labels = y)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [70]:
session = tf.Session()
session.run(tf.global_variables_initializer())

## 抓 Batch 來訓練
---

In [73]:
# 每次抓 100 來訓練
batch_size = 100

In [74]:
def batch_data(batch_size):
    batch_x = []
    batch_y = []
    for i in range(0, batch_size):
        randomSize = len(FeatureDataList)
        randomIndex = random.randint(0, randomSize)
        batch_x.append(FeatureDataLista[randomIndex])
        batch_y.append(AnsDataList[randomIndex])
    return batch_x, batch_y

In [75]:
def optimize(num_iterations):
    data_x, data_y = batch_data(batch_size)
    session.run(optimizer, feed_dict={x: data_x, y: data_y})

In [ ]:
optimize